In [1]:
import itertools
import json
import math
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

from glob import glob
from pathlib import Path

INTERVAL = 15
# TARGETFILENAME = 'sorted_rfcm_results_similarity.csv'
SOURCEFILENAME = 'sorted_rfcm_results.csv'

有沒有做 reconstruction 都一樣

In [2]:
anomaly_src = ["212.136.43.87", "212.136.43.86", "196.135.158.151", "94.31.10.141", "212.136.37.68", "202.64.138.203", "197.67.136.111", "197.67.136.114", "212.136.45.198", "192.205.17.73", "172.36.161.123", "171.26.178.147", "94.73.100.101", "219.184.185.217", "172.36.184.245", "171.148.229.203", "171.148.229.203", "196.135.171.114", "212.205.124.182", "94.73.100.101", "213.139.30.85", "204.195.4.72", "75.193.32.24", "80.247.190.194", "88.82.56.168", "206.150.69.212", "206.150.69.212", "156.15.129.204", "73.203.208.228", "218.239.122.63", "187.168.11.99", "94.82.58.70", "197.207.99.18", "197.207.99.142"]
anomaly_dst = ["64.232.149.222", "156.15.6.0", "212.136.46.159", "212.136.46.159", "213.21.54.56", "172.36.161.123", "213.139.28.46", "172.36.73.76", "212.136.37.68", "83.8.20.216", "196.197.236.47", "204.71.67.83", "171.131.219.68", "171.131.219.68", "212.136.44.72", "171.26.178.162", "171.131.27.40", "171.147.252.192", "212.136.38.56", "212.136.38.56", "212.136.38.55", "212.136.38.55", "213.139.30.85", "171.131.27.91", "171.131.27.91", "219.184.185.220", "156.15.175.217", "156.15.175.217", "212.136.45.198", "212.136.38.54", "212.136.38.54", "212.136.43.64", "213.139.37.2", "156.15.175.211", "156.15.175.211", "156.15.175.211", "172.36.184.245", "172.36.0.174", "156.15.6.247", "156.15.6.247", "156.15.114.72", "206.206.160.107", "172.36.179.27", "156.15.194.169", "156.15.194.169", "156.15.194.169", "212.136.43.87", "88.88.56.12"]

suspicious_src = ["172.36.29.104", "212.136.46.159", "212.136.46.159", "212.136.43.64", "213.21.49.75", "183.233.222.211", "0.215.192.227", "91.83.104.72", "170.203.128.98", "88.217.0.197", "188.225.212.47", "53.173.206.119", "63.0.168.9", "199.70.4.229", "78.164.232.25", "43.155.83.116", "215.94.105.84", "44.161.6.251", "189.130.57.97", "42.201.185.218", "76.110.104.34", "81.4.107.243", "45.97.255.238", "83.94.228.238", "199.215.156.20", "75.205.190.245", "51.52.16.151", "34.118.116.240", "213.125.25.200", "213.125.25.200", "212.150.144.14", "216.2.208.4", "195.72.129.128", "202.44.64.199", "171.131.219.40", "156.15.175.211", "156.15.175.211", "156.15.175.211", "152.238.233.76", "152.238.233.76", "171.131.173.141", "171.131.27.150", "88.88.42.9", "98.182.37.4", "172.36.184.137", "196.19.57.49", "83.8.20.51", "199.134.100.73", "199.134.100.72", "206.150.69.212", "206.150.69.212", "204.212.136.170", "196.158.241.79", "152.238.232.82", "101.125.161.66", "43.17.130.41", "75.193.32.24", "172.36.198.37", "172.36.174.199", "171.148.21.21", "171.131.27.41", "156.15.6.247", "216.105.40.74", "212.136.34.175", "212.136.47.1", "197.207.99.17", "200.213.42.46", "91.152.17.145", "91.152.21.78", "91.7.91.18", "101.28.19.161", "91.7.93.19", "172.36.187.74", "171.131.176.6", "171.131.176.6", "179.168.26.227", "179.168.26.227", "80.205.68.236", "80.205.68.236", "213.193.189.66"]
suspicious_dst = ["95.139.240.207", "197.67.136.111", "197.67.136.114", "192.168.96.4", "79.190.32.85", "91.196.191.12", "171.131.173.141", "204.214.53.137", "204.214.53.137", "171.131.32.151", "171.131.32.151", "156.15.52.69", "156.15.6.239", "199.200.73.82", "156.15.33.34", "156.15.33.34", "156.15.33.34", "213.129.205.65", "213.129.205.65", "213.129.205.65", "156.15.6.239", "156.15.175.211", "156.15.175.211", "156.15.175.211", "212.136.38.57", "212.136.38.57", "171.26.178.147", "213.139.37.78", "212.136.45.59", "196.135.171.114", "44.232.121.238", "213.129.204.69", "212.136.47.145", "156.15.175.210", "2.0.198.225", "172.36.189.140", "156.15.1.173", "212.136.43.163", "44.100.78.247", "46.154.243.117", "94.82.58.22", "80.205.94.252", "156.15.4.146", "156.15.4.146", "93.70.70.235", "77.162.21.28", "213.34.225.178", "101.28.40.10", "172.36.198.37", "171.26.178.48", "171.26.178.48"]

features = ["packets", "bytes", "flows", "bytes_packets", "flows_(bytes_packets)", "nDstIP", "nSrcPort", "nDstPort"]

In [3]:
with open('timeseries_feature/interval_15_src_feature/types.json') as f:
    types = json.load(f)
df = pd.read_csv('timeseries_feature/interval_15_src_feature/sorted_rfcm_results.csv', index_col=0, dtype=types)

df["avg"] = df.mean(axis=1)
df.sort_values(by="avg", ascending=False, inplace=True)

first_column = df.pop('avg')
df.insert(0, 'avg', first_column)
df.reset_index(inplace=True)

df[df.ip.isin(anomaly_src)]

,ip,avg,bytes-bytes_packets,bytes-flows,bytes-flows_(bytes_packets),bytes-nDstIP,bytes-nDstPort,bytes-nSrcPort,bytes_packets-flows_(bytes_packets),bytes_packets-nDstIP,bytes_packets-nDstPort,bytes_packets-nSrcPort,flows-bytes_packets,flows-flows_(bytes_packets),flows-nDstIP,flows-nDstPort,flows-nSrcPort,flows_(bytes_packets)-nDstIP,flows_(bytes_packets)-nDstPort,flows_(bytes_packets)-nSrcPort,nDstIP-nDstPort,nDstIP-nSrcPort,nSrcPort-nDstPort,packets-bytes,packets-bytes_packets,packets-flows,packets-flows_(bytes_packets),packets-nDstIP,packets-nDstPort,packets-nSrcPort
31,212.136.43.87,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
46,212.136.43.86,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
53,88.82.56.168,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
57,212.205.124.182,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
69,192.205.17.73,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
75,212.136.45.198,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
102,171.26.178.147,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
104,94.82.58.70,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
134,187.168.11.99,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
138,212.136.37.68,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2


In [4]:
with open('timeseries_feature/interval_15_src_feature_reconstructed_STL_combined/types.json') as f:
    types = json.load(f)
df = pd.read_csv('timeseries_feature/interval_15_src_feature_reconstructed_STL_combined/sorted_rfcm_results.csv', index_col=0, dtype=types)

df["avg"] = df.mean(axis=1)
df.sort_values(by="avg", ascending=False, inplace=True)

first_column = df.pop('avg')
df.insert(0, 'avg', first_column)
df.reset_index(inplace=True)

df[df.ip.isin(anomaly_src)]

,ip,avg,bytes-bytes_packets,bytes-flows,bytes-flows_(bytes_packets),bytes-nDstIP,bytes-nDstPort,bytes-nSrcPort,bytes_packets-flows_(bytes_packets),bytes_packets-nDstIP,bytes_packets-nDstPort,bytes_packets-nSrcPort,flows-bytes_packets,flows-flows_(bytes_packets),flows-nDstIP,flows-nDstPort,flows-nSrcPort,flows_(bytes_packets)-nDstIP,flows_(bytes_packets)-nDstPort,flows_(bytes_packets)-nSrcPort,nDstIP-nDstPort,nDstIP-nSrcPort,nSrcPort-nDstPort,packets-bytes,packets-bytes_packets,packets-flows,packets-flows_(bytes_packets),packets-nDstIP,packets-nDstPort,packets-nSrcPort
31,212.136.43.87,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
46,212.136.43.86,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
53,88.82.56.168,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
57,212.205.124.182,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
69,192.205.17.73,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
75,212.136.45.198,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
102,171.26.178.147,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
104,94.82.58.70,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
134,187.168.11.99,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
138,212.136.37.68,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2


In [5]:
with open('timeseries_feature/interval_15_src_feature_reconstructed_STL_detrend/types.json') as f:
    types = json.load(f)
df = pd.read_csv('timeseries_feature/interval_15_src_feature_reconstructed_STL_detrend/sorted_rfcm_results.csv', index_col=0, dtype=types)

df["avg"] = df.mean(axis=1)
df.sort_values(by="avg", ascending=False, inplace=True)

first_column = df.pop('avg')
df.insert(0, 'avg', first_column)
df.reset_index(inplace=True)

df[df.ip.isin(anomaly_src)]

,ip,avg,bytes-bytes_packets,bytes-flows,bytes-flows_(bytes_packets),bytes-nDstIP,bytes-nDstPort,bytes-nSrcPort,bytes_packets-flows_(bytes_packets),bytes_packets-nDstIP,bytes_packets-nDstPort,bytes_packets-nSrcPort,flows-bytes_packets,flows-flows_(bytes_packets),flows-nDstIP,flows-nDstPort,flows-nSrcPort,flows_(bytes_packets)-nDstIP,flows_(bytes_packets)-nDstPort,flows_(bytes_packets)-nSrcPort,nDstIP-nDstPort,nDstIP-nSrcPort,nSrcPort-nDstPort,packets-bytes,packets-bytes_packets,packets-flows,packets-flows_(bytes_packets),packets-nDstIP,packets-nDstPort,packets-nSrcPort
31,212.136.43.87,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
46,212.136.43.86,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
53,88.82.56.168,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
57,212.205.124.182,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
69,192.205.17.73,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
75,212.136.45.198,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
102,171.26.178.147,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
104,94.82.58.70,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
134,187.168.11.99,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
138,212.136.37.68,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2


In [6]:
with open('timeseries_feature/interval_15_src_feature_reconstructed_STL_seasonal/types.json') as f:
    types = json.load(f)
df = pd.read_csv('timeseries_feature/interval_15_src_feature_reconstructed_STL_seasonal/sorted_rfcm_results.csv', index_col=0, dtype=types)

df["avg"] = df.mean(axis=1)
df.sort_values(by="avg", ascending=False, inplace=True)

first_column = df.pop('avg')
df.insert(0, 'avg', first_column)
df.reset_index(inplace=True)

df[df.ip.isin(anomaly_src)]

,ip,avg,bytes-bytes_packets,bytes-flows,bytes-flows_(bytes_packets),bytes-nDstIP,bytes-nDstPort,bytes-nSrcPort,bytes_packets-flows_(bytes_packets),bytes_packets-nDstIP,bytes_packets-nDstPort,bytes_packets-nSrcPort,flows-bytes_packets,flows-flows_(bytes_packets),flows-nDstIP,flows-nDstPort,flows-nSrcPort,flows_(bytes_packets)-nDstIP,flows_(bytes_packets)-nDstPort,flows_(bytes_packets)-nSrcPort,nDstIP-nDstPort,nDstIP-nSrcPort,nSrcPort-nDstPort,packets-bytes,packets-bytes_packets,packets-flows,packets-flows_(bytes_packets),packets-nDstIP,packets-nDstPort,packets-nSrcPort
31,212.136.43.87,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
46,212.136.43.86,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
53,88.82.56.168,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
57,212.205.124.182,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
69,192.205.17.73,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
75,212.136.45.198,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
102,171.26.178.147,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
104,94.82.58.70,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
134,187.168.11.99,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
138,212.136.37.68,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2


In [7]:
with open('timeseries_feature/interval_15_src_feature_reconstructed_STL_trend/types.json') as f:
    types = json.load(f)
df = pd.read_csv('timeseries_feature/interval_15_src_feature_reconstructed_STL_trend/sorted_rfcm_results.csv', index_col=0, dtype=types)

df["avg"] = df.mean(axis=1)
df.sort_values(by="avg", ascending=False, inplace=True)

first_column = df.pop('avg')
df.insert(0, 'avg', first_column)
df.reset_index(inplace=True)

df[df.ip.isin(anomaly_src)]

,ip,avg,bytes-bytes_packets,bytes-flows,bytes-flows_(bytes_packets),bytes-nDstIP,bytes-nDstPort,bytes-nSrcPort,bytes_packets-flows_(bytes_packets),bytes_packets-nDstIP,bytes_packets-nDstPort,bytes_packets-nSrcPort,flows-bytes_packets,flows-flows_(bytes_packets),flows-nDstIP,flows-nDstPort,flows-nSrcPort,flows_(bytes_packets)-nDstIP,flows_(bytes_packets)-nDstPort,flows_(bytes_packets)-nSrcPort,nDstIP-nDstPort,nDstIP-nSrcPort,nSrcPort-nDstPort,packets-bytes,packets-bytes_packets,packets-flows,packets-flows_(bytes_packets),packets-nDstIP,packets-nDstPort,packets-nSrcPort
31,212.136.43.87,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
46,212.136.43.86,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
53,88.82.56.168,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
57,212.205.124.182,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
69,192.205.17.73,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
75,212.136.45.198,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
102,171.26.178.147,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
104,94.82.58.70,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
134,187.168.11.99,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
138,212.136.37.68,2.357143,2,2,2,2,2,2,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,5,2,3,2,2,2
